In [1]:
import sys

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

print(sys.executable)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'\nDevice: {device}')

/home/mehrzad/anaconda3/envs/dbpedia/bin/python

Device: cuda:0


In [2]:
checkpoint="Salesforce/codegen25-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_4bit=True, device_map="auto")


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
text = "write a DBpedia SPARQL query to find the population of Italy "
input_ids = tokenizer(text, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

write a DBpedia SPARQL query to find the population of Italy 
- The population of Italy is a property of the "http://dbpedia.org/ontology/Population" class
- The population of Italy is also a property of the "http://dbpedia.org/resource/Italy" resource
- The population of Italy is located in the "http://dbpedia.org/resource/Italy" resource
- The population of Italy is located in the "http://dbpedia.org/ontology/population" property
- The population of Italy is located in the "http://dbpedia.


In [3]:
text = "write a simple SPARQL query to find capital of Germany using DBpedia ontology"
input_ids = tokenizer(text, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


write a simple SPARQL query to find capital of Germany using DBpedia ontology

### Response: The following SPARQL query can be used to find the capital of Germany from the DBpedia ontology:

```sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#


In [6]:
prompt = """
Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"answer"=
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"answer"=
SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
  FILTER (lang(?capital) = 'en')
}

"question"= what is the capital of Italy?
"answer"=
SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
  FILTER (lang(?capital) = 'en')
}

"question"= what is the capital of Italy?


In [18]:
prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

#


In [10]:
prompt = """
Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"answer"=
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"answer"=
SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
  FILTER (lang(?capital) = 'en')
}

"question"= what is the capital of Italy?
"answer"=
SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
  FILTER (lang(?capital) = 'en')
}

"question"= what is the capital of Italy?


In [21]:
# check for one-shot generalization

prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?
SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the population of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?
SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the population of Italy?
SELECT ?population WHERE {
  dbr:Italy dbo:populationTotal ?population .
}


#


In [22]:
# check for one-shot generalization

prompt = """
# write a SPARQL query

# what is the capital of Japan?
SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the population of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query

# what is the capital of Japan?
SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the population of Italy?
SELECT ?population WHERE {
  dbr:Italy dbo:populationTotal ?population .
}


#


In [ ]:
<|endoftext|><sep>

In [34]:
# check for one-shot generalization

prompt = """
# what is the capital of Japan?
SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}<|endoftext|>


# what is the population of Italy?
<mask_1><|endoftext|><sep>
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# what is the capital of Japan?
SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}<|endoftext|>


# what is the population of Italy?
<mask_1><|endoftext|><sep>
#


In [36]:
tokenizer.decode(inputs.input_ids[0])

'\n# what is the capital of Japan?\nSELECT ?capital WHERE {\n  dbr:Japan dbo:capital ?capital .\n}<|endoftext|>\n\n\n# what is the population of Italy?\n<mask_1><|endoftext|><sep>\n'

In [41]:
tokenizer.encode(['<sep>'])

[50317]

In [37]:
inputs.input_ids[0]

tensor([  198,     2,   644,   318,   262,  3139,   286,  2869,    30,   198,
        46506,  5633, 27544, 33411,  1391,   198,   220,   288,  1671,    25,
        16504,   288,  2127,    25, 27544,  5633, 27544,   764,   198,    92,
        50256,   628,   198,     2,   644,   318,   262,  3265,   286,  8031,
           30,   198, 51199, 50256, 50317,   198], device='cuda:0')

In [17]:
prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the capital of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the capital of Italy?

SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
}


#


In [44]:
prompt = """
# SPARQL query from question

# question: what is the dbo:capital of dbr:Japan?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# SPARQL query from question

# question: what is the dbo:capital of dbr:Japan?

#


In [17]:
prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the capital of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# what is the capital of Italy?

SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
}


#


In [45]:
prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# people born in the capital of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# people born in the capital of Italy?

SELECT ?person WHERE {
  ?person dbo:placeOfBirth dbr:Italy .
  dbr:Italy dbo:capital ?capital .
  ?person dbo:placeOfBirth ?capital .
}


#


In [46]:
prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# people born in the capital of Italy before 2000?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# people born in the capital of Italy before 2000?

SELECT ?person WHERE {
  ?person dbo:birthPlace dbr:Italy .
  ?person dbo:birthDate ?birthDate .
  FILTER(YEAR(?birthDate) < 2000)
}


#


In [48]:
prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# List of 10 people born in the capital of Italy before 2000?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# List of 10 people born in the capital of Italy before 2000?

SELECT ?person WHERE {
  ?person dbo:birthPlace dbr:Italy .
  ?person dbo:birthDate ?birthDate .
  FILTER (?birthDate < "2000-01-01"^^xsd:date)
}
LIMIT 10


#


In [21]:
prompt = """# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?
SELECT ?capital WHERE {dbr:Japan dbo:capital ?capital .}


# people born in the capital of Italy before 2000?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))

# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?
SELECT ?capital WHERE {dbr:Japan dbo:capital ?capital .}


# people born in the capital of Italy before 2000?
SELECT ?person WHERE {
  ?person dbo:birthPlace dbr:Italy .
  ?person dbo:birthDate ?birthDate .
  FILTER(YEAR(?birthDate) < 2000)
}


#


In [49]:
prompt = """
# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# who killed Ceaser?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?

SELECT ?capital WHERE {
  dbr:Japan dbo:capital ?capital .
}


# who killed Ceaser?

SELECT ?person WHERE {
  dbr:Ceaser dbo:deathPlace ?place .
  ?place dbo:deathDate "1861-01-01"^^xsd:date .
  ?person dbo:death ?place .
  ?person dbo:birthDate "1861-01-01"^^xsd:date .
}


#


In [4]:
## adjusting EOS token to control the length of the output 

eos_token = tokenizer.eos_token
eos_token_id = tokenizer.eos_token_id

print(f'End of Sequence token:\t{eos_token}\t with id={eos_token_id}\n')

End of Sequence token:	<|endoftext|>	 with id=50256



In [8]:
prompt = """
Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX dbc: <http://dbpedia.org/resource/Category:><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"answer"=
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


Compelete "answer" for "question" which is a SPARQL query towards the DBpedia knowledge base using these prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX dbc: <http://dbpedia.org/resource/Category:><|endoftext|>

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }<|endoftext|>

"question"= what is the capital of Italy?
"answer"=
SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
  FILTER (lang(?capital) = 'en')
}

"question"= what is the capital of Italy?
"answer"=
SELECT ?capital WHERE {
  dbr:Italy dbo:capital ?capital .
  FILTER (lang(?capital) = 'en')
}

"question"= what is the capital of Italy?


In [11]:
prompt = """
"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }

"question"= what is the capital of Italy?
"answer"=
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }

"question"= what is the capital of Italy?
"answer"=

### Response: "question"= what is the capital of Italy?
"answer"= SELECT ?capital WHERE { dbr:Italy dbo:capital ?capital . }

"question"= what is the capital of Japan?
"answer"= SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }

"question"= what is the capital of France?
"answer"= SELECT ?capital WHERE { dbr:France d


In [42]:
inputs['input_ids']

tensor([[  198,     2,   644,   318,   262,  3139,   286,  2869,    30,   198,
         46506,  5633, 27544, 33411,  1391,   198,   220,   288,  1671,    25,
         16504,   288,  2127,    25, 27544,  5633, 27544,   764,   198,    92,
         50256,   628,   198,     2,   644,   318,   262,  3265,   286,  8031,
            30,   198, 51199, 50256, 50317,   198]], device='cuda:0')

In [13]:
if eos_token_id not in inputs['input_ids'][0]:
    print(f'\nNo EOS token found in tokenized input')


No EOS token found in tokenized input


In [17]:
prompt = """# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?
SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }


# what was the population of France in 2000?
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


# write a SPARQL query with following prefixes:
# PREFIX dbr: <http://dbpedia.org/resource/>
# PREFIX dbo: <http://dbpedia.org/ontology/>

# what is the capital of Japan?
SELECT ?capital WHERE { dbr:Japan dbo:capital ?capital . }


# what was the population of France in 2000?
SELECT ?population WHERE { dbr:France dbo:populationTotal ?population . FILTER(?year="2000") }

#


In [64]:
print(truncate_at_eom(tokenizer.decode(outputs[0]))[len(prompt):])

In [62]:
def truncate_at_eom(text):
    return text.split('<eom>', 1)[0].strip()

# Example
output = "The capital of France is Paris. For more information, visit our website.<eom> random characters or irrelevant text here..."
truncated_output = truncate_at_eom(output)
print(truncated_output)


The capital of France is Paris. For more information, visit our website.


In [3]:
prompt = """
Write a SPARQL query to generate 
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


if eos_token_id not in inputs['input_ids'][0]:
    print(f'\n\nNo EOS token found in tokenized input')

/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')



Write a SPARQL query to generate 
a table that shows the top 10 countries with the highest population density, 
calculated by dividing the number of people living in each country by the 
area of that country in square kilometers. The table should also include the 
population of each country.

### Response: Here's the SPARQL query that generates the table you requested:

```
PREFIX rdf: <http://www.w3.org/1999/02/22-rd


NameError: name 'eos_token_id' is not defined

In [40]:
def format(prefix, suffix):
    return prefix + "<mask_1>" + suffix + "<|endoftext|>" + "<sep>" + "<mask_1>"


prefix = "def hello_world():\n    "
suffix = "    return name"
text = format(prefix, suffix)



inputs = tokenizer.encode_plus(text, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0])[len(text):])




# input_ids = tokenizer(text, return_tensors="pt").input_ids
# generated_ids = model.generate(input_ids, max_length=128)
# print(tokenizer.decode(generated_ids[0], skip_special_tokens=False)[len(text):])


    return render_template('<eom><|endoftext|><filename><mask_1>/<mask_2>/<mask_3>.py
#


def print_hello_world():
<mask_1><|endoftext|><sep><mask_1>


In [56]:
prefix = "def print_hello_world():\n    "
suffix = ""
text = format(prefix, suffix)
input_ids = tokenizer(text, return_tensors="pt").input_ids

print(text)
# generated_ids = model.generate(input_ids, max_length=500)
# print(tokenizer.decode(generated_ids[0], skip_special_tokens=False))
   



def print_hello_world():
    <mask_1><|endoftext|><sep><mask_1>


In [58]:
input_ids

tensor([[ 4299,  3601,    62, 31373,    62,  6894, 33529,   198, 50284, 51199,
         50256, 50317, 51199]])

In [59]:
tokenizer.decode(input_ids[0])

'def print_hello_world():\n    <mask_1><|endoftext|><sep><mask_1>'

In [60]:
inputs = tokenizer.encode_plus(text, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100, attention_mask=inputs['attention_mask'])
print(tokenizer.decode(outputs[0]))


def print_hello_world():
    <mask_1><|endoftext|><sep><mask_1>
    print("Hello World<eom><|endoftext|><reponame><mask_1>/<mask_2>-<mask_3>
import<mask_4>
import<mask_5>
import<mask_6>
import<mask_7>
import<mask_8>
import<mask_9>
import<mask_10>
import<mask_11>
import<mask_12>
import<mask_13>
import<mask_14>
import<mask_15>
import<mask_16>
import<mask_17>
import<mask_18>
import<mask_19>
import<mask_20>
import<mask_21>
import<mask_22>
import<mask_23>
import<mask_24>
import<mask_25>
import<mask_26>
import<mask_27>
import<mask_28>
import<mask_29>
import<mask_30>
import<mask_31>
import


#
# Evaluation
### Initial BLEU score for QALD-9-plus set


In [53]:
import json

with open('data/qald_9_plus_test_dbpedia.json', 'r') as f:
    test_data = json.load(f)
    
test_data['questions'][0]

{'id': '99',
 'question': [{'language': 'en',
   'string': 'What is the time zone of Salt Lake City?'},
  {'language': 'de', 'string': 'In welcher Zeitzone liegt Salt Lake City?'},
  {'language': 'de', 'string': 'Was ist die Zeitzone von Salt Lake City?'},
  {'language': 'ru', 'string': 'Какой часовой пояс в Солт-Лейк-Сити'},
  {'language': 'ru',
   'string': 'В каком часовом поясе расположен Солт-Лейк-Сити?'},
  {'language': 'uk', 'string': 'Який часовий пояс у Солт-Лейк Сіті?'},
  {'language': 'lt', 'string': 'Kokia Solt Leik Sičio laiko zona?'},
  {'language': 'be', 'string': 'Які гадзінны пояс у Солт-Лэйк-Сіці'},
  {'language': 'lt', 'string': 'Kokia laiko juosta yra Solt Leik Sityjes'},
  {'language': 'ba', 'string': 'Ниндей вакыт поясы Солт-Лейк-Ситила'},
  {'language': 'es',
   'string': '¿En qué zona horaria se encuentra Salt Lake City?',
   'keywords': 'Salt Lake City,  zona horaria'}],
 'query': {'sparql': 'PREFIX res: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedi

In [58]:
# extracting pairs of question/sparql    
    
nl_questions, sparql_queries =[],[]

for data in test_data['questions']:
    nl_questions.append(
        next((item['string'] for item in data['question'] if item['language'] == 'en'), None)
    )
    sparql_queries.append(data['query']['sparql'])
    

print(f'{len(nl_questions)} pairs extracted from test set: ', "\n")
print(nl_questions[0])
print('\n', sparql_queries[0].replace(">", '>\n'))

150 pairs extracted from test set:  

What is the time zone of Salt Lake City?

 PREFIX res: <http://dbpedia.org/resource/>
 PREFIX dbp: <http://dbpedia.org/property/>
 SELECT DISTINCT ?uri WHERE { res:Salt_Lake_City <http://dbpedia.org/ontology/timeZone>
 ?uri }


In [94]:
# not all sparql queries are with explicit prefixes at the begining

ls=[]

for query in sparql_queries:
    ls.append(query.split()[0])
    
print(set(ls))
print(ls.count('PREFIX'))

{'SELECT', 'ASK', 'PREFIX'}
88


##
For now, let's avoid dealing with 

In [105]:
import re

def extract_prefixes(query):
    pattern = re.compile(r'(PREFIX\s+[\w\-]+:\s+<[^>]+>)')
    return pattern.findall(query)


                
def format_prompt(nl_questions, sparql_queries):
    prompts = []
    base_prompt = "# write a SPARQL query with following prefixes:\n"

    for question, query in zip(nl_questions, sparql_queries):
        if query.split()[0] == 'PREFIX':
            prefix_lines = extract_prefixes(query)
            prompt_prefixes = '\n'.join(f"# {prefix}" for prefix in prefix_lines)
            full_prompt = base_prompt + prompt_prefixes + "\n\n# " + question

            prompts.append((full_prompt, question, query))

    return prompts

        
prompts= format_prompt(nl_questions,sparql_queries)
print(prompts[1][0])
# print(prompts[1][1],'\n')
# print(prompts[1][2])

# write a SPARQL query with following prefixes:
# PREFIX dct: <http://purl.org/dc/terms/>
# PREFIX dbc: <http://dbpedia.org/resource/Category:>

# Who killed Caesar?


In [107]:
print(sparql_queries[1])

PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?uri dct:subject dbc:Assassins_of_Julius_Caesar }


In [95]:
len(prompts)

88

In [75]:
import re

def find_unique_uri(queries):
    uri_pattern = re.compile(r'<(http://[^>]+)>')
    http_addresses = set()
    
    for query in queries:
        uris = uri_pattern.findall(query)
        http_addresses.update(uris)

    return http_addresses

find_unique_uri(sparql_queries)

{'http://dbpedia.org/',
 'http://dbpedia.org/class/yago/',
 'http://dbpedia.org/class/yago/City108524735',
 'http://dbpedia.org/class/yago/WikicatAmericanVegans',
 'http://dbpedia.org/class/yago/WikicatJapaneseMusicalInstruments',
 'http://dbpedia.org/class/yago/WikicatStatesOfTheUnitedStates',
 'http://dbpedia.org/ontology/',
 'http://dbpedia.org/ontology/Airline',
 'http://dbpedia.org/ontology/Airport',
 'http://dbpedia.org/ontology/Animal',
 'http://dbpedia.org/ontology/Astronaut',
 'http://dbpedia.org/ontology/Automobile',
 'http://dbpedia.org/ontology/BasketballPlayer',
 'http://dbpedia.org/ontology/Book',
 'http://dbpedia.org/ontology/City',
 'http://dbpedia.org/ontology/Country',
 'http://dbpedia.org/ontology/Film',
 'http://dbpedia.org/ontology/Library',
 'http://dbpedia.org/ontology/Person',
 'http://dbpedia.org/ontology/Place',
 'http://dbpedia.org/ontology/PoliticalParty',
 'http://dbpedia.org/ontology/Politician',
 'http://dbpedia.org/ontology/River',
 'http://dbpedia.org/o

In [77]:
def convert_to_prefix_format(query):
    prefix_mappings = {
        'http://dbpedia.org/resource/': 'res',
        'http://dbpedia.org/ontology/': 'dbo',
        'http://dbpedia.org/property/': 'dbp',
        'http://xmlns.com/foaf/0.1/': 'foaf',
        'http://www.w3.org/2001/XMLSchema#': 'xsd',        
        'http://www.w3.org/2002/07/owl#': 'owl',
        'http://www.w3.org/1999/02/22-rdf-syntax-ns#': 'rdf',
    }
    
    uris = set(re.findall(r'<([^>]+)>', query))
    
    prefix_statements = []
    for base_uri, prefix in prefix_mappings.items():
        for uri in uris:
            if uri.startswith(base_uri):
                local_name = uri[len(base_uri):]
                query = query.replace(uri, f"{prefix}:{local_name}")
                if f"PREFIX {prefix}:" not in prefix_statements:
                    prefix_statements.append(f"PREFIX {prefix}: <{base_uri}>")
    
    return ' '.join(prefix_statements) + ' ' + query